In [98]:
#nltk.download()

In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.stem import PorterStemmer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score

In [100]:
df=pd.read_csv(r'E:\spam_classifier\sms+spam+collection\SMSSpamCollection',sep='\t',names=['label','text'])

In [101]:
corpus=[]
ps=PorterStemmer()
l=WordNetLemmatizer()
for i in range(0,len(df)):
    review=re.sub('[^a-zA-Z]',' ',df['text'][i])
    review=review.lower()
    review=review.split()

    review=[l.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)
cv=CountVectorizer(max_features=2500)
x=cv.fit_transform(corpus).toarray()
le=LabelEncoder()
y=le.fit_transform(df['label'])


In [102]:
smote=SMOTE(random_state=42)
x_sampl,y_sampl=smote.fit_resample(x,y)

In [103]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

In [104]:
model=MultinomialNB()
model.fit(xtrain,ytrain)

MultinomialNB()

In [105]:
ypred=model.predict(xtest)

In [106]:
confusion_matrix(ytest,ypred)

array([[954,  12],
       [  6, 143]], dtype=int64)

In [107]:
recall_score(ytest,ypred)

0.959731543624161

In [108]:
accuracy_score(ytest,ypred)

0.9838565022421525